In [2]:
from sklearn.ensemble import IsolationForest
import pandas as pd
import json
import numpy as np

data = pd.read_csv('DataForRouting.csv')
data = data.dropna(subset=['LATITUDE', 'LONGITUDE', 'POSTAL_CODE'])
data = data.to_dict(orient='index')

with open('configurations.json', 'r') as fp:
    config = json.load(fp)

print('clients: ', len(data))

clients:  1238


In [3]:
# Usefull Functions
def anomalyDetection(ironmongeries, contamination: float):
    X = np.array([ [i['lng'], i['lat']] for i in ironmongeries])
    anomaly_algorithm = IsolationForest(contamination=contamination)
    y_pred = anomaly_algorithm.fit(X).predict(X)

    newList = []
    for idx, value in enumerate(y_pred):
        if value == 1:
            newList.append(ironmongeries[idx])

    return newList

In [7]:
# For Each Configuration Build Routing Data

# Service
import hunterRoutingService as hrs
allSoluions = []
for zone in config:
    PCs = zone['PC']
    subData = {
        'id':zone['Location'],
        'ironmongeries' : [ {
            'lat': data[i]['LATITUDE'], 
            'lng': data[i]['LONGITUDE'], 
            'id': data[i]['ID'],
            'type': 'Decrece',
            'coordinates': [ data[i]['LATITUDE'], data[i]['LONGITUDE'] ] 
            } for i in data if data[i]['POSTAL_CODE'] in PCs ]
    }
    iniLen = len(subData['ironmongeries'])
    contP = 1-zone['Routes']*30/iniLen
    subData['ironmongeries'] = anomalyDetection(subData['ironmongeries'],contP)

    HZ = hrs.RoutingHunterService()
    sol = HZ.hunterRouting(subData)

    allSoluions.append(sol)

    print(zone['Location'], iniLen, len(subData['ironmongeries']), len(sol['routes']))
    

Barrios Unidos 39 30 1
Bosa 103 90 3
Engativa 111 90 3
Fontibon 36 30 1
Kennedy 108 90 3
Puente Aranda 55 30 1
Rafael Uribe Uribe 42 30 1
San Cristobal 63 30 1
Soacha 87 60 2
Suba 149 90 4
Usaquen 49 30 1


In [16]:
data = pd.read_csv('DataForRouting.csv', index_col = 'ID')
data = data.dropna(subset=['LATITUDE', 'LONGITUDE', 'POSTAL_CODE'])
data = data.to_dict(orient='index')

for i in allSoluions:
    count = 1
    for j in i['routes']:
        name = i['id'] + '_' + str(count)
        featureList = []
        linestring = []

        orderRoute = 1
        for client in j:
            # Individual Points
            coors = [data[client['id']]['LONGITUDE'] , data[client['id']]['LATITUDE']]
            linestring.append(coors)

            featureList.append({
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": coors
                },
                "properties": {
                    "id": client['id'],
                    "name": data[client['id']]['NAME'],
                    "order": orderRoute
                }
            })
            orderRoute += 1

        # Full Route
        featureList.append({
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": linestring
            },
            "properties": {
                "Route": name
            }
        })

        with open(name+'.geojson', 'w') as fp:
            json.dump({
                "type": "FeatureCollection",
                "features": featureList
            }, fp)
        count += 1
        
